In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

- How many layers,
- MaxPooling,
- 1x1 Convolutions,
- 3x3 Convolutions,
- Receptive Field,
- SoftMax,
- Learning Rate,
- Kernels and how do we decide the number of kernels?
- Batch Normalization,
- Image Normalization,
- Position of MaxPooling,
- Concept of Transition Layers,
- Position of Transition Layer,
- DropOut
- When do we introduce DropOut, or when do we know we have some overfitting
- The distance of MaxPooling from Prediction,
- The distance of Batch Normalization from Prediction,
- When do we stop convolutions and go ahead with a larger kernel or some other alternative (which we have not yet covered)
- How do we know our network is not going well, comparatively, very early
Batch Size, and Effects of batch size

In [31]:
class Net_comp(nn.Module):
  
  def __init__(self):
    super(Net_comp, self).__init__()
    
    self.b1_C = 8
    self.b2_C = 14
    self.b3_C = 24
    self.b4_C = 32
    self.dp_p = 0.1
    
    self.block1_l = []
    self.block1_l.append(nn.Conv2d(1, self.b1_C, 3, padding='same', bias=False))
    self.block1_l.append(nn.ReLU())
    self.block1_l.append(nn.BatchNorm2d(self.b1_C))
    self.block1_l.append(nn.Dropout(self.dp_p))
    self.block1_l.append(nn.Conv2d(self.b1_C, self.b1_C, 3, padding='same', bias=False))
    self.block1_l.append(nn.ReLU())
    self.block1_l.append(nn.BatchNorm2d(self.b1_C))
    self.block1_l.append(nn.Dropout(self.dp_p))
    self.block1 = nn.Sequential(*self.block1_l)
    
    self.trblock1 = nn.Sequential(nn.MaxPool2d(2, 2), 
                                  nn.Conv2d(self.b1_C, self.b1_C, 1, bias=False))
    

    self.block2_l = []
    self.block2_l.append(nn.Conv2d(self.b1_C, self.b2_C, 3, padding='same', bias=False))
    self.block2_l.append(nn.ReLU())
    self.block2_l.append(nn.BatchNorm2d(self.b2_C))
    self.block2_l.append(nn.Dropout(self.dp_p))
    self.block2_l.append(nn.Conv2d(self.b2_C, self.b2_C, 3, padding='same', bias=False))
    self.block2_l.append(nn.ReLU())
    self.block2_l.append(nn.BatchNorm2d(self.b2_C))
    self.block2_l.append(nn.Dropout(self.dp_p))
    self.block2 = nn.Sequential(*self.block2_l)
    
    self.trblock2 = nn.Sequential(nn.MaxPool2d(2, 2), 
                                  nn.Conv2d(self.b2_C, self.b2_C, 1, bias=False))

    self.block3_l = []
    self.block3_l.append(nn.Conv2d(self.b2_C, self.b3_C, 3, padding='same', bias=False))
    self.block3_l.append(nn.ReLU())
    self.block3_l.append(nn.BatchNorm2d(self.b3_C))
    self.block3_l.append(nn.Dropout(self.dp_p))
    self.block3_l.append(nn.Conv2d(self.b3_C, self.b3_C, 3, padding='same', bias=False))
    self.block3_l.append(nn.ReLU())
    self.block3_l.append(nn.BatchNorm2d(self.b3_C))
    self.block3_l.append(nn.Dropout(self.dp_p))
    self.block3 = nn.Sequential(*self.block3_l)
    
    self.trblock3 = nn.Sequential(nn.MaxPool2d(2, 2), 
                                  nn.Conv2d(self.b3_C, self.b3_C, 1, bias=False))
    
    self.block4_l = []
    self.block4_l.append(nn.Conv2d(self.b3_C, self.b4_C, 3, padding='same', bias=False))
    self.block4_l.append(nn.ReLU())
    self.block4_l.append(nn.BatchNorm2d(self.b4_C))
    self.block4_l.append(nn.Dropout(self.dp_p))
    self.block4_l.append(nn.Conv2d(self.b4_C, 10, 1, bias=False))
    self.block4_l.append(nn.AvgPool2d(3, 2))
    self.block4 = nn.Sequential(*self.block4_l)

  def forward(self, x):
    x = self.block1(x)
    x = self.trblock1(x)
    x = self.block2(x)
    x = self.trblock2(x)
    x = self.block3(x)
    x = self.trblock3(x)
    x = self.block4(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)

In [8]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net_comp().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             576
              ReLU-6            [-1, 8, 28, 28]               0
       BatchNorm2d-7            [-1, 8, 28, 28]              16
           Dropout-8            [-1, 8, 28, 28]               0
         MaxPool2d-9            [-1, 8, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]              64
           Conv2d-11           [-1, 14, 14, 14]           1,008
             ReLU-12           [-1, 14, 14, 14]               0
      BatchNorm2d-13           [-1, 14, 14, 14]              28
          Dropout-14           [-1, 14,

<ipython-input-28-56f086c9c1c0>:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [20]:
torch.manual_seed(1)
batch_size = 512

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 195151735.77it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 35482042.71it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 111257724.24it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 22465246.19it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [21]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [32]:
model = Net_comp().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1, verbose=True)

for epoch in range(1, 20):
    print(f'[INFO] Epoch #{epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    # Invoke the scheduler
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-01.
[INFO] Epoch #1


  0%|          | 0/118 [00:00<?, ?it/s]<ipython-input-31-e3a1790c39f2>:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.037816811352968216 batch_id=117: 100%|██████████| 118/118 [00:16<00:00,  7.19it/s]



Test set: Average loss: 0.0960, Accuracy: 9691/10000 (97%)

Adjusting learning rate of group 0 to 1.0000e-01.
[INFO] Epoch #2


loss=0.0258304625749588 batch_id=117: 100%|██████████| 118/118 [00:16<00:00,  7.17it/s]



Test set: Average loss: 0.0591, Accuracy: 9812/10000 (98%)

Adjusting learning rate of group 0 to 1.0000e-01.
[INFO] Epoch #3


...


Test set: Average loss: 0.0169, Accuracy: 9943/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.
[INFO] Epoch #18


loss=0.030348142609000206 batch_id=117: 100%|██████████| 118/118 [00:16<00:00,  6.97it/s]



Test set: Average loss: 0.0164, Accuracy: 9946/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.
[INFO] Epoch #19


loss=0.03849955275654793 batch_id=117: 100%|██████████| 118/118 [00:16<00:00,  7.27it/s]



Test set: Average loss: 0.0170, Accuracy: 9943/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.
